In [21]:
## Fetching Data at Census Tract Level based on Location using GEO PANDAS for taxi data 
## and identifying which list of census tracts under each zone

In [1]:
##Install GEO-PANADS
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [41]:
census = r"/Users/kavyavempati/Desktop/Kavya/Capstone/cb_2022_36_tract_500k"   ##Census Tract Shape Files
taxi_zone= r"/Users/kavyavempati/Desktop/Kavya/Capstone/taxi_zones- shape files" ## Taxi Zone Shape Files
excel_output1 =r"/Users/kavyavempati/Desktop/Kavya/Capstone/census_wZones.xlsx"  ## Census Vs Zones
excel_output2 = r"/Users/kavyavempati/Desktop/Kavya/Capstone/zones_wCensus.xlsx"  ## Zones Vs Census

In [3]:
##Create Dataframe for Census file
census_df=gpd.read_file(census) 

In [4]:
##Create Dataframe for Taxi Zones file
taxi_zones_df = gpd.read_file(taxi_zone)

In [5]:
print(census_df.crs, taxi_zones_df.crs)

GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]] PROJCS["NAD_1983_StatePlane_New_York_Long_Island_FIPS_3104_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",40.6666666666667],PARAMETER["standard_parallel_2",41.0333333333333],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["Foot_US",0.304800609601219],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [6]:
#Converting to same as zone projection to align them
census_df = census_df.to_crs(taxi_zones_df.crs)

In [7]:
print(census_df.crs, taxi_zones_df.crs)

PROJCS["NAD_1983_StatePlane_New_York_Long_Island_FIPS_3104_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",40.6666666666667],PARAMETER["standard_parallel_2",41.0333333333333],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["Foot_US",0.304800609601219],AXIS["Easting",EAST],AXIS["Northing",NORTH]] PROJCS["NAD_1983_StatePlane_New_York_Long_Island_FIPS_3104_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",40.6666666666667]

In [8]:
#Same as taxi zone , making census data as well same as zone
clipped_census = gpd.clip(census_df, taxi_zones_df)

In [9]:
#Converting multi parts to single parts to get seperate centriods
clipped_census_MS = clipped_census.explode(ignore_index=True)

In [10]:
#Creating Centroid for each 
census_pts = clipped_census_MS.copy()
census_pts['geometry']=census_pts['geometry'].centroid

In [11]:
#Zone With CensusTract -One To Many
census_pts_sj = gpd.sjoin(census_pts, taxi_zones_df, how='left')
census_pts_sj.to_excel(excel_output1, index=False)

In [44]:
taxi_zones_df_sj.head

<bound method NDFrame.head of      OBJECTID  Shape_Leng  Shape_Area            zone  LocationID    borough  \
0           1    0.116357    0.000782  Newark Airport           1        EWR   
1           2    0.433470    0.004866     Jamaica Bay           2     Queens   
1           2    0.433470    0.004866     Jamaica Bay           2     Queens   
1           2    0.433470    0.004866     Jamaica Bay           2     Queens   
1           2    0.433470    0.004866     Jamaica Bay           2     Queens   
..        ...         ...         ...             ...         ...        ...   
262       263    0.037017    0.000066  Yorkville West         263  Manhattan   
262       263    0.037017    0.000066  Yorkville West         263  Manhattan   
262       263    0.037017    0.000066  Yorkville West         263  Manhattan   
262       263    0.037017    0.000066  Yorkville West         263  Manhattan   
262       263    0.037017    0.000066  Yorkville West         263  Manhattan   

         

In [45]:
taxi_zones_df_sj.columns

Index(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID', 'borough',
       'geometry', 'old_index_right', 'STATEFP', 'COUNTYFP', 'TRACTCE',
       'AFFGEOID', 'GEOID', 'NAME', 'NAMELSAD', 'STUSPS', 'NAMELSADCO',
       'STATE_NAME', 'LSAD', 'ALAND', 'AWATER'],
      dtype='object')

In [12]:
#CensusTract with Zone - Many to One
taxi_zones_df_sj = gpd.sjoin(taxi_zones_df, census_pts, how='left')
taxi_zones_df_sj .to_excel(excel_output2, index=False)